In [1]:
import tensorflow as tf
from joblib import load
from utils.model_analysis import  plot_shap_over_time
from utils.data_loader import load_as_maps

/Users/jakobmeggendorfer/Documents/CAU/Masterarbeit/master-thesis/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "../../outputs/u-net/2025-09-23_15-21"    
dataset_id = "exp1"

# load model
model = tf.keras.models.load_model(model_path + "/model.keras")

# load and prepare data
X_test, Y_test = load_as_maps(start_year=2014, end_year=2015, datasets=[dataset_id])
map_mask = X_test[0,:,:, 11] == 1

scaler = load(model_path + '/scaler.pkl')
n_samples, h, w, n_features = X_test.shape
X_test_flat = X_test.reshape(-1,n_features)
X_test_scaled_flat = scaler.transform(X_test_flat)
X_test = X_test_scaled_flat.reshape(n_samples, h, w, n_features)

In [3]:
# plot feature importance over time per basin

feature_names = ['SST', 'SAL', 'ice_frac', 'mixed_layer_depth', 'heat_flux_down', 'water_flux_up', 'stress_X', 'stress_Y', 'currents_X', 'currents_Y','mask','month-sin','month-cos']

print(X_test.shape)
print(map_mask.shape)

mean_abs_shap, names = plot_shap_over_time(
    model, X_test, map_mask, feature_names=feature_names,
    bg_per_time=10,            # small background per time step
    max_samples_per_time=20, # subsample for speed (optional)
    # batch_size=4096,
    region_id='ARCTIC',
    type='SEASONAL'
)

(24, 167, 360, 13)
(167, 360)
12


IndexError: list index out of range